Requirements

In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import math
import os
import pandas as pd

In [79]:

def patch_thresholding(gray,patch_size,const):
    
    result_mean = np.copy(gray)
    
    height,width=gray.shape
    
    origin = math.floor(patch_size/2)
    padded = np.pad(gray, origin, mode='constant', constant_values=0)
    
    index_x = index_y = 0
    for i in tqdm(range(origin, height+origin)):
        for j in range(origin, width+origin):
            patch = padded[i-origin:i+origin+1, j-origin:j+origin+1]
            local_mean = np.mean(patch)
            
            threshold_mean = local_mean - const
            
            if padded[i,j] >= threshold_mean:
               result_mean[index_y, index_x] = 1
            else:
                result_mean[index_y, index_x] = 0 
             
            index_x = min(index_x+1, width)
        index_y = min(index_y+1, height)
        index_x = 0
        
    normalized_result_mean = np.clip((result_mean - result_mean.min()) / (result_mean.max() - result_mean.min()), 0, 1)
    
    return normalized_result_mean

In [ ]:
root = r'\Fundus image' # give directory here
images = os.listdir(root)

for file_name in images:
    full_path = os.path.join(root,file_name)
    
    image = cv2.imread(full_path,cv2.IMREAD_GRAYSCALE)
    
    result= patch_thresholding(image,7,5)
    
    plt.figure(figsize=(15,5))
    plt.subplot(1,2,1)
    plt.imshow(image,cmap='gray')
    plt.axis("off")
    
    plt.subplot(1,2,2)
    plt.imshow(result,cmap='gray')
    plt.axis("off")
    plt.show()
    break
    

In [19]:
def adaptive_threshold_image(image,epsilon):
    result_mean = result_median = np.zeros_like(image)
    
    
    T1 = int(np.mean(image))
    while True:
        
        G1 = image[image < T1]
        G2 = image[image > T1]
        
        M1 = np.mean(G1)
        M2 = np.mean(G2)
        
        T2 = int((M1+M2)/2)
        
        if abs(T2-T1) < epsilon:
            break
        else:
            T1 = T2
            
    _, result_mean = cv2.threshold(image, T2, 255, cv2.THRESH_BINARY)
    
    
    T2 = np.median(image)    
    
    _, result_median = cv2.threshold(image, T2, 255, cv2.THRESH_BINARY)
        
    return result_mean,result_median

PLOTTING VEINS

In [ ]:

root = r'\Fundus image'
images = os.listdir(root)

for file_name in images:
    full_path = os.path.join(root,file_name)
    
    image = cv2.imread(full_path,cv2.IMREAD_GRAYSCALE)
    
    result,_ = adaptive_threshold_image(image,5)
    
    adaptive_thresh = cv2.adaptiveThreshold(image, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 5, 5)
    
    _, binary = cv2.threshold(image, 170, 255, cv2.THRESH_BINARY)
    
    # Plot the results side by side
    fig, axes = plt.subplots(1, 3, figsize=(12, 4))

    axes[0].imshow(image, cmap='gray')
    axes[0].set_title('Original Image')
    axes[0].axis('off')

    axes[1].imshow(adaptive_thresh, cmap='gray')
    axes[1].set_title('Adaptive Threshold buitin')
    axes[1].axis('off')
    
    axes[2].imshow(binary, cmap='gray')
    axes[2].set_title('basic threshold')
    axes[2].axis('off')

    plt.tight_layout()
    plt.show()

    #break




PREPROCESSING

In [3]:
def stretch_contrast(image):
    
    min_val = np.percentile(image, 15)
    max_val = np.percentile(image, 99)
    
    stretched_image = np.clip(image, min_val, max_val)
    
    stretched_image = (stretched_image - min_val) / (max_val - min_val) * 255
    
    stretched_image = stretched_image.astype(np.uint8)
    
    return stretched_image

In [ ]:

root = r'\Fundus image'
images = os.listdir(root)

subjects = []

for i in range(len(images)):
    
    
    full_path = os.path.join(root,images[i])
    
    print(full_path)
    
    orignal = cv2.imread(full_path,cv2.IMREAD_GRAYSCALE)
    
    enhanced = stretch_contrast(orignal)
    
    _, binary = cv2.threshold(enhanced, 254, 255, cv2.THRESH_BINARY)
    
    adaptive_threshold = cv2.adaptiveThreshold(enhanced, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 11, 2)
    
    subjects.append(binary)
    
    #veins = patch_thresholding(enhanced,7,5)
    
    # Plot the results side by side
    fig, axes = plt.subplots(1, 4, figsize=(12, 4))

    axes[0].imshow(orignal, cmap='gray')
    axes[0].set_title('Original Image')
    axes[0].axis('off')

    axes[1].imshow(enhanced, cmap='gray')
    axes[1].set_title('enhanced image')
    axes[1].axis('off')
    
    axes[2].imshow(binary, cmap='gray')
    axes[2].set_title('basic thresholding')
    axes[2].axis('off')
    
    axes[3].imshow(adaptive_threshold, cmap='gray')
    axes[3].set_title('adaptive thresholding')
    axes[3].axis('off')
    
    plt.tight_layout()
    plt.show()

    #break
    




CCA

In [ ]:
results_x = []
results_y = []

for subject in subjects:
    
    kernel = [
    [0, 0, 1, 0, 0],
    [0, 1, 1, 1, 0],
    [1, 1, 1, 1, 1],
    [0, 1, 1, 1, 0],
    [0, 0, 1, 0, 0]
]

    kernel_array = np.array(kernel,dtype=np.uint8)
    
    eroded_image = cv2.erode(subject, kernel_array, iterations=1)
    dilated_image = cv2.dilate(eroded_image, kernel_array, iterations=1)
    
    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(dilated_image, connectivity=8)

    # num_labels: Total number of labels (including background)
    # labels: Image where each pixel is assigned a label
    # stats: Statistics for each connected component (e.g., area, bounding box)
    # centroids: Centroid coordinates for each connected component

    largest_label = np.argmax(stats[1:, cv2.CC_STAT_AREA]) + 1

    area = stats[largest_label, cv2.CC_STAT_AREA]
    centroid = centroids[largest_label].astype(int)
    print(f"Largest connected component: Label={largest_label}, Area={area}, Centroid={centroid}")

    results_x.append(centroids[largest_label][0].astype(int))
    results_y.append(centroids[largest_label][1].astype(int))
    
    largest_area_mask = np.where(labels == largest_label, 255, 0).astype(np.uint8)

    labeled_image = cv2.cvtColor(subject, cv2.COLOR_GRAY2BGR)
    labeled_image = cv2.applyColorMap((labels * 255 / num_labels).astype('uint8'), cv2.COLORMAP_JET)
    
    fig, axes = plt.subplots(1, 4, figsize=(12, 4))

    axes[0].imshow(subject, cmap='gray')
    axes[0].set_title('thresholded')
    axes[0].axis('off')


    axes[1].imshow(dilated_image, cmap='gray')
    axes[1].set_title('after opening')
    axes[1].axis('off')
    
    largest_area_image = cv2.bitwise_and(dilated_image, dilated_image, mask=largest_area_mask)
    
    axes[2].imshow(labeled_image, cmap='gray')
    axes[2].set_title('all objects')
    axes[2].axis('off')
    
    axes[3].imshow(largest_area_image, cmap='gray')
    axes[3].set_title('optic-disc')
    axes[3].axis('off')
    
    plt.tight_layout()
    plt.show()

    #break
    
print(results_x,results_y)



In [ ]:

df = pd.read_csv(r'\optic_disc_centres.csv')

csv_file_names = df.iloc[:, 0].values
csv_x = df.iloc[:, 1].values
csv_y = df.iloc[:, 2].values

for i in range(len(csv_file_names)):
    print(csv_file_names[i], csv_x[i],csv_y[i])




PERFORMANCE METRICS

In [ ]:
errors = []

for i in range(len(results_y)):
    error = ( (csv_x[i]-results_x[i])**2 + (csv_y[i]-results_y[i])**2 )**0.5
    print(error)
    errors.append(error)

In [ ]:

df['File name'] = csv_file_names
df['actual_x'] = csv_x
df['actual_y'] = csv_y
df['Results_x'] = results_x
df['Results_y'] = results_y
df['Errors'] = errors

df.to_csv(r'F:\NOTES\Season 3.2\DIP\Assignment-2 data\Assignment-2\output_file2.csv', index=False)

print("Data has been written to 'output_file.csv'")